In [ ]:
from flask import Flask, request, render_template
from keras.models import load_model
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import keras.backend as k
    
#Create an app object using the Flask class
app = Flask(__name__)

#Load the trained models
modelMLP = pickle.load(open('MLP.pkl','rb'))
modelCNN = load_model('cnn')
    
@app.route('/')
def home():
    #send us to application.html
    return render_template('Main.html') 

@app.route('/process', methods=['POST']) #POST: send HTML form data to the server.
def process():
    file_path = 'static/data.wav'
    file = request.files.get('file')
    
    if file.filename == '':
        return render_template('Main.html')
            
    file.save(file_path)
    
    x = extract_feature(file_path, mfcc=True, chroma=True, mel=True)
    x = np.array(x).reshape(1, x.shape[0], 1)
    keras_function = k.function([modelCNN.input], [modelCNN.layers[4].output])
    cnn_testing = keras_function([x])[0]
    output = modelMLP.predict_proba(cnn_testing)[0]
    
    for i in range(len(output)):
        output[i] = round(output[i] * 100, 2)
        
    return render_template('Main.html', real_time_on_off="false", calm=output[0], disgust=output[1],
                          fearful=output[2], happy=output[3])


def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result


    
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Dec/2022 22:22:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2022 22:22:55] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\tandi\AppData\Local\Temp\ipykernel_17268\2690650921.py:60: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ...  9.4604492e-04
  8.5449219e-04  8.8500977e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
127.0.0.1 - - [20/Dec/2022 22:23:23] "POST /process HTTP/1.1" 200 -
C:\Users\tandi\AppData\Local\Temp\ipykernel_17268\2690650921.py:60: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
127.0.0.1 - - [20/Dec/2022 22:23:37] "POST /process HTTP/1.1" 200 -
